# Multi Color Light Curves  applied LSSTComCamSim : DZP on CCD


- author Sylvie Dagoret-Campagne
- creation date 2024-05-03
- last update 2024-06-02


  
``Quantum Graphs``: https://tigress-web.princeton.edu/~lkelvin/pipelines/ 


https://tigress-web.princeton.edu/~lkelvin/pipelines/current/drp_pipe/LSSTComCamSim/nightly-validation-ops-rehearsal-3/

``CalibrateTask``: 
https://github.com/lsst/pipe_tasks/blob/main/python/lsst/pipe/tasks/calibrate.py#L392-L399


``Shemas``:
https://dm.lsst.org/sdm_schemas/browser/

In [ ]:
from lsst.daf.butler import Butler

import astropy.units as u
import numpy as np 
import pandas as pd
pd.set_option("display.max_columns", None)
from astropy.time import Time

import scipy.stats

import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm

import lsst.afw.display.rgb as afwRgb
import lsst.afw.image as afwImage
import lsst.geom as geom
import pickle
from astropy.stats import SigmaClip

In [ ]:
#xx-small
#x-small
#small
#medium
#large
#x-large
#xx-large

plt.rcParams["figure.figsize"] = (10,6)
plt.rcParams["axes.labelsize"] = 'x-large'
plt.rcParams['axes.titlesize'] = 'x-large'
plt.rcParams['xtick.labelsize']= 'x-large'
plt.rcParams['ytick.labelsize']= 'x-large'

In [ ]:
from shapely.geometry import Point as shapelyPoint
from shapely.geometry.polygon import Polygon as shapelyPolygon

In [ ]:
def is_in_polygon(x,y,ll,lr,ur,ul):
    point = shapelyPoint(x, y)
    polygon = shapelyPolygon([ll, lr, ur, ul])
    return polygon.contains(point)

In [ ]:
def angle_in_range(alpha, lower, upper):
    return (alpha - lower) % 360 <= (upper - lower) % 360

In [ ]:
def isradec_inbbox_radec(ra,dec,ramin,ramax,decmin,decmax):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    ra: ra in degree
    dec: dec in degree
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    Bool
    
    """


    flag_ra = angle_in_range(ra,ramin,ramax)
    flag_dec = angle_in_range(dec,decmin,decmax)

    flag = flag_ra and flag_dec
    return flag


In [ ]:
def get_corners_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    corners_ra, corners_dec in decimal degrees
    """

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    corners_ra = []
    corners_dec = []
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec

In [ ]:
saveDir = "./lc_output_dmrehearsal2024"
doSaveFigs = False  # set to True if you actually want to save the figures in the above dir

In [ ]:
# Set some selections for reliable data
minNumMatches = 3
minSnCalibFlux =  100

In [ ]:
# INSERT YOUR collection and tract
# for rehearsal use collection 2 which have CCDvisit
butlerRoot = "/repo/embargo"

collection1 = 'LSSTComCamSim/runs/nightlyvalidation/20240402/d_2024_03_29/DM-43612'
collection2 = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
collection3 = 'LSSTComCamSim/runs/nightlyvalidation/20240404/d_2024_03_29/DM-43612'
collectionn = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
#collections = [collection1,collection2,collection3]
collections = [collection2]
collectionStr = collectionn.replace("/", "_")
instrument = 'LSSTComCamSim'
skymapName = "ops_rehersal_prep_2k_v1"
where_clause = "instrument = \'" + instrument+ "\'"
#3533 : No matching visitId
tract = 2494
#tract = 7445
#tract = 9880
#band = "r"
suptitle = collectionStr + f" inst = {instrument} tract = {tract}"

calibFluxStr = "apFlux_12_0_instFlux"
calibFluxErrStr = "apFlux_12_0_instFluxErr"
calibFluxMagStr = "apFlux_12_0_instMag"
calibFluxMagErrStr = "apFlux_12_0_instMagErr"

calibFluxCalStr = "apFlux_12_0_calFlux"
calibFluxCalErrStr = "apFlux_12_0_calFluxErr"
calibFluxCalMagStr = "apFlux_12_0_calMag"
calibFluxCalMagErrStr = "apFlux_12_0_calMagErr"

file_output_selectedsources = f"lightcurves-dmrehearsal2024_04_03_tract{tract}.csv"
fullname_file_output_selectedsources = os.path.join(saveDir,file_output_selectedsources)

## Initiate butler from variables set above

In [ ]:
# Initiate butler from variables set above
butler = Butler(butlerRoot, collections=collections, instrument=instrument, skymap=skymapName)
camera = butler.get("camera", instrument=instrument)
skymap = butler.get("skyMap")
print("camera_name = {}".format(camera.getName()))
print("collections = {}".format(collections))

## load isolated_star_sources

In [ ]:
# Try to get the Schema
all_tracts = []
data_product = "isolated_star_sources"
datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=collections, where= where_clause)
for i, ref in enumerate(datasetRefs):
    print(i,ref.dataId)
    all_tracts.append(ref.dataId["tract"])
    #butler_data = butler.get(ref)

all_tracts = sorted(all_tracts)
print("all tracts : ", all_tracts)

#if not isinstance(butler_data, pd.core.frame.DataFrame):
#    print(butler_data.getSchema())

## isolated_star_sources Keeping all bands here, but keep the tract selection

The main starting point is the table of isolated_star_sources which has been constructed in step 2a from a catalog
Note that isolated star sources are associated to an object (a static starobject)
Here the work is doneband by band.
It is a good idea to work band by band.

In [ ]:
# Load in isolated_star_sources and trim to band of interest and select the tract
isolatedStarSourcesFull = butler.get("isolated_star_sources", tract=tract)
#isolatedStarSourcesFull = isolatedStarSourcesFull[isolatedStarSourcesFull["band"] == band]

In [ ]:
tract

In [ ]:
len(isolatedStarSourcesFull)

In [ ]:
# Just to have a look at what's in the catalog:
isolatedStarSourcesFull[isolatedStarSourcesFull.index == 0]

In [ ]:
isolatedStarSourcesFull.head()

In [ ]:
isolatedStarSourcesFull.tail()

In [ ]:
isolatedStarSourcesFull.columns

In [ ]:
print(sorted(isolatedStarSourcesFull["visit"].unique()))

In [ ]:
# Just to have a look at what's in the catalog:
isolatedStarSourcesFull[isolatedStarSourcesFull.index == 0]

In [ ]:
print("calibFluxStr = {},,minSnCalibFlux = {}, minNumMatches = {}".format(calibFluxStr,minSnCalibFlux,minNumMatches))

## Statistics on detectors in isolated_star_sources

In [ ]:
ser_isolatedstar_perdetectorband_size = isolatedStarSourcesFull.groupby(["detector","band"]).size()

In [ ]:
ser_isolatedstar_perdetectorband_size

In [ ]:
df_isolatedstar_perdetectorband = ser_isolatedstar_perdetectorband_size.unstack(level=1)
df_isolatedstar_perdetectorband 

In [ ]:
from itertools import cycle, islice
my_colors = list(islice(cycle(['g', 'r', 'orange']), None, len(df_isolatedstar_perdetectorband)))
df_isolatedstar_perdetectorband.plot.bar(color=my_colors,rot=0,title=f"Number of isolated sources per detector and per filter in tract {tract}") 

In [ ]:
NDET=9
import seaborn as sns

In [ ]:
from matplotlib.colors import ListedColormap
cmap = ListedColormap(sns.color_palette("hls", NDET))
all_det_colors = [cmap.colors[idx] for idx in range(NDET)]

In [ ]:
cmap

In [ ]:
df_isolatedstar_perdetectorband = ser_isolatedstar_perdetectorband_size.unstack(level=0)
df_isolatedstar_perdetectorband 

In [ ]:
# not working 
#my_colors_cycle = list(islice(cycle(all_det_colors), None, len(df_isolatedstar_perdetectorband)))
#my_colors_cycle = list(islice(cycle(all_det_colors),None,3))

In [ ]:
# not working
#import itertools
#my_colors_cycle = itertools.cycle(all_det_colors)
#my_colors_cycle = itertools.cycle(cmap)

In [ ]:
df_isolatedstar_perdetectorband.plot.bar(color=all_det_colors + all_det_colors + all_det_colors,rot=0,title=f"Number of isolated sources per detector and per filter in tract {tract}") 

In [ ]:
my_colors = list(islice(cycle(['g', 'r', 'orange']), None, NDET))
ser_isolatedstar_perdetectorband_size.plot.bar(x="detector",color=my_colors,title=f"Number of isolated sources per detector and per filter in tract {tract}")

## Select a number of visits with enough good sources but in all bands here

- extract the list of known objects

In [ ]:
calibFluxStr

In [ ]:
minSnCalibFlux

In [ ]:
# Trim the isolated_star_sources cat to those PSF S/N > minSnCalibFlux
# and number of matches sources > minNumMatches.
objIndexListFull = list(set(isolatedStarSourcesFull["obj_index"].values))

# Select the bright stars above a minimum of S/N ratio
if minSnCalibFlux is not None:
    snCalibFlux = isolatedStarSourcesFull[calibFluxStr]/isolatedStarSourcesFull[calibFluxStr + "Err"]
    snMask = snCalibFlux > minSnCalibFlux
    isolatedStarSourcesMinSn = isolatedStarSourcesFull[snMask].copy(deep=True)
else:
    isolatedStarSourcesMinSn = isolatedStarSourcesFull.copy(deep=True)

# object index with obj index in above selected bright selected star
objIndexListMinSn = list(set(isolatedStarSourcesMinSn["obj_index"].values))

# requires that the object is associated to several sources (at least minNumMatches sources)
# loop on object having sources with S/N aboe selection 
objIndexList = []
for objIndex in objIndexListMinSn:
    objData = isolatedStarSourcesMinSn[isolatedStarSourcesMinSn["obj_index"] == objIndex]
    if len(objData) >= minNumMatches:
        objIndexList.append(objIndex)
numTrimmed = len(objIndexListFull) - len(objIndexList)

mask = []
for objIndex, visit in zip(isolatedStarSourcesMinSn["obj_index"], isolatedStarSourcesMinSn["visit"]):
    if objIndex in objIndexList:
        mask.append(True)
    else:
        mask.append(False)
isolatedStarSources = isolatedStarSourcesMinSn[mask].copy(deep=True)

# Select a number of visits according quality criteria
visitList = list(set(isolatedStarSources["visit"].values))

if minSnCalibFlux is not None:
    print("Trimmed isolated_star_sources catalog to S/N {} > {} (leaving N={} matched sources from original {})".format(
        calibFluxStr, minSnCalibFlux, len(isolatedStarSources), len(isolatedStarSourcesFull)))
print("Trimmed isolated_star_sources catalog to objecst with nMatches >= {} (leaving {} objects from {}).".format(
    minNumMatches, len(objIndexList), len(objIndexListFull)))

In [ ]:
sorted_visitList = sorted(visitList)
#sorted_visitList 

In [ ]:
def convertVisitToDatestr(visit):

    num = visit//100_000
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return datestr

In [ ]:
def convertVisitToMJD(visit):
    return Time(convertVisitToDatestr(visit)).mjd

In [ ]:
sorted_visits_mjd = list(map(convertVisitToMJD, sorted_visitList))

## sourceTable_visit

This is the main output of FGCM calibration which has selected a number of sources. These sources were originally selected from isolated sources.

In [ ]:
# Extra columns to load from sourceTable_visit catalogs
names = ["psfFlux", "psfFluxErr", "psfFlux_flag", "psfFlux_apCorr", "psfFlux_apCorrErr",
         "extendedness", "detect_isPrimary", "deblend_skipped",
         "gaussianFlux", "gaussianFluxErr", "gaussianFlux_flag",
         "localPhotoCalib", "localPhotoCalibErr", "localPhotoCalib_flag"]

## Note that some sourceTable_visit not found for some visits in isolated_star_sources 

In [ ]:
#visitList

Note here that the sourceId is the same in isolated_source_table and sourceTable

In [ ]:
# Load in the sourceTable_visit catalogs to get the psfFlux and other useful info per source.
# Columns loaded are those in names above.
# The selected sources from the sourceTable_visit will go in a list inside the sourceCatalog

sourceCatalogs = []
nSourceDict = {}
visitsToRemove = []

# loop over all selected visit and load sourceTable_visit visit per visit
for visit in visitList:
    try:
        sourceCatalogOrig = butler.get("sourceTable_visit", visit=visit, parameters={"columns": names})
        #select all the sources that have a "detect_isPrimary" flag in the sourceTable_visit
        primaryCat = sourceCatalogOrig[sourceCatalogOrig["detect_isPrimary"]].copy(deep=True)
        # count the number of sources having S/N > 5 in this visit
        nSourceDict[visit] = sum(primaryCat["psfFlux"]/primaryCat["psfFluxErr"] > 5)
        #keep those sources hat have a "detect_isPrimary" flag in the sourceTable_visit in a list of list of sources per visits
        sourceCatalogs.append(primaryCat)
    except LookupError:
        print("sourceTable_visit not found for visit {} which was in isolated_star_sources".format(visit))
        isolatedStarSources = isolatedStarSources[isolatedStarSources["visit"] != visit].copy(deep=True)
        visitsToRemove.append(visit)
visitList = list(set(visitList) - set(visitsToRemove))

# Do the minNumMatches cut again since visits may have been removed due to lack of sourceTable_visit
print("Number of objects with >=3 matches before sourceTable_visit exsistence check: {}".format(len(objIndexList)))
objIndexListAllNew = list(set(isolatedStarSources["obj_index"].values))
objIndexList = []
for objIndex in objIndexListAllNew:
    objData = isolatedStarSources[isolatedStarSources["obj_index"] == objIndex]
    if len(objData) >= minNumMatches:
        objIndexList.append(objIndex)
mask = []
for objIndex, visit in zip(isolatedStarSources["obj_index"], isolatedStarSources["visit"]):
    if objIndex in objIndexList:
        mask.append(True)
    else:
        mask.append(False)
isolatedStarSources = isolatedStarSources[mask].copy(deep=True)
visitList = list(set(isolatedStarSources["visit"].values))
print("Number of objects with >=3 matches after sourceTable_visit existence check: {}".format(len(objIndexList)))

# Final action associate the source in the sourceTable if the visit to the source in the isolated_source_table 
# on source sourceid and then concat all merged 
dataJoined = pd.concat(sourceCatalogs).merge(isolatedStarSources, on="sourceId", how="inner")

In [ ]:
len(sourceCatalogs)

In [ ]:
#nSourceDict

In [ ]:
#visitList

In [ ]:
visitsToRemove 

## Will start to work on dataJoined which associate selected "isolated_star_sources" in the selected band and visits the good calibration in "sourceTable_visit"

### retrieve CCD Visit Table

- it provides airmass and initial zero-point

In [ ]:
ccdVisitTable = butler.get("ccdVisitTable")

In [ ]:
 ccdVisitTable

In [ ]:
ccdVisitTable_subset = ccdVisitTable.iloc[:20][["visitId","detector","band"]]
ccdVisitTable_subset

In [ ]:
#ccdVisitTable_noindex = ccdVisitTable.reset_index()
#ccdVisitTable_noindex
#ccdVisitTable_noindex.groupby(["ccdVisitId","detector"]).size()

In [ ]:
ccdVisitTable.groupby(["visitId","ccdVisitId","detector"]).size()

Thus one ccvisit is associated to one visit id and a detector id
wheras the visitId has

In [ ]:
for visit in visitList:
    if visit in ccdVisitTable["visitId"].values:
        print(ccdVisitTable[ccdVisitTable["visitId"] == visit]["zeroPoint"])
        print(ccdVisitTable[ccdVisitTable["visitId"] == visit]["detector"])
        break

### Calculate Airmass and Ellipticity

In [ ]:
ccdVisitTable["airmass"] = ccdVisitTable["zenithDistance"].apply(lambda x: 1/np.cos(np.pi/180.*x))
ccdVisitTable["medianE"] = np.sqrt(ccdVisitTable["psfStarDeltaE1Median"] ** 2.0 + ccdVisitTable["psfStarDeltaE2Median"] ** 2.0)

In [ ]:
# Collect useful columns from ccdVisitTable
# This is wrong when we have several detectors per visit
if 0:
    visitsCCDToRemove = []

    #Note there is one value per detector
    psfSigmaDict = {}
    skyBgDict = {}
    skyNoiseDict = {}
    expTimeDict = {}
    expMidptMjdDict = {}
    medianEDict = {}
    psfStarScaledDeltaSizeScatterDict = {}
    astromOffsetStdDict = {}
    psfTraceRadiusDeltaDict = {}
    zeroPointDict = {}
    airmassDict = {}
    seeingDict = {}

    for visit in visitList:
        if visit in ccdVisitTable["visitId"].values:
            psfSigmaDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["psfSigma"].values[0]
            skyBgDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["skyBg"].values[0]
            skyNoiseDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["skyNoise"].values[0]
            expTimeDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["expTime"].values[0]
            expMidptMjdDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["expMidptMJD"].values[0]
            medianEDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["medianE"].values[0]
            psfStarScaledDeltaSizeScatterDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["psfStarScaledDeltaSizeScatter"].values[0]
            astromOffsetStdDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["astromOffsetStd"].values[0]
            psfTraceRadiusDeltaDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["psfTraceRadiusDelta"].values[0]
            zeroPointDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["zeroPoint"].values[0]
            airmassDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["airmass"].values[0]
            seeingDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["seeing"].values[0]

        else:
            #raise RuntimeError("ERROR: visit {} not found in ccdVisitTable".format(visit))
            print("ERROR: CCD visit {} not found in ccdVisitTable".format(visit))
            visitsCCDToRemove.append(visit) 
            psfSigmaDict[visit] = np.nan
            skyBgDict[visit] = np.nan
            skyNoiseDict[visit] = np.nan
            expTimeDict[visit] = np.nan
            expMidptMjdDict[visit] = np.nan
            medianEDict[visit] = np.nan
            psfStarScaledDeltaSizeScatterDict[visit] = np.nan
            astromOffsetStdDict[visit] = np.nan
            psfTraceRadiusDeltaDict[visit] = np.nan
            zeroPointDict[visit] = np.nan
            airmassDict[visit] = np.nan
            seeingDict[visit] = np.nan

In [ ]:
#visitList

In [ ]:
#visitsToRemove 

In [ ]:
#visitsCCDToRemove

In [ ]:
# Just to have a look at what's in the catalog:
#ccdVisitTable[ccdVisitTable.index == ccdVisitTable.index[:5]] 

### Plot visits versus time

In [ ]:
ccdVisitTable_subset = ccdVisitTable[["visitId","band","zeroPoint","psfSigma","skyBg","skyNoise","seeing","expTime","expMidptMJD","obsStart","airmass","medianE","psfStarScaledDeltaSizeScatter",
                                      "astromOffsetStd","psfTraceRadiusDelta",
                                      "llcra","llcdec","ulcra","ulcdec","urcra","urcdec","lrcra","lrcdec"]]

#### compute nightObs and datetime time

In [ ]:
ccdVisitTable_subset["nightObs"] = ccdVisitTable_subset.apply(lambda x: x['visitId']//100_000 - 50000000, axis=1)
ccdVisitTable_subset["Time"] = pd.to_datetime(ccdVisitTable_subset['obsStart'])

In [ ]:
ccdVisitTable_g = ccdVisitTable_subset[ccdVisitTable_subset.band== "g"]
ccdVisitTable_r = ccdVisitTable_subset[ccdVisitTable_subset.band== "r"]
ccdVisitTable_i = ccdVisitTable_subset[ccdVisitTable_subset.band== "i"]
#ccdVisitTable_z = ccdVisitTable_subset[ccdVisitTable_subset.band== "z"]
#ccdVisitTable_y = ccdVisitTable_subset[ccdVisitTable_subset.band== "y"]

In [ ]:
ccdVisitTable_subset.head()

In [ ]:
ccdVisitTable_subset["removed"] = ccdVisitTable_subset["visitId"].apply(lambda x: True if x in visitsToRemove else False) 

In [ ]:
ccdVisitTable_subset_removed = ccdVisitTable_subset[ccdVisitTable_subset["removed"]] 

In [ ]:
ccdVisitTable_subset_removed 

#### Thus the visit IN isolated-star_sources and MISSING in sourceTable are also NOT in ccdVisitTable !!!

In [ ]:
ccdVisitTable_subset.tail()

In [ ]:
# function of time conversion
from astropy.time import Time

def dt_to_mjd(dt):
    t = Time(dt)
    return t.to_value('mjd','float')

def mjd_to_dt(mjd):
    tm = Time(mjd, format='mjd')
    # Convert to datetime
    utc =tm.to_datetime()
    return utc

In [ ]:
dt = ccdVisitTable_subset.iloc[0]['Time']
dt

In [ ]:
type(dt)

In [ ]:
mjd = dt_to_mjd(dt)
mjd

In [ ]:
tm = mjd_to_dt(mjd)

In [ ]:
#ax.secondary_xaxis?

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")

fig,axs = plt.subplots(3,1,figsize=(16,10))
ax1,ax2,ax3 = axs
ccdVisitTable_g.plot(x="Time",y="zeroPoint",marker='+',c="g",lw=0.0,ax=ax1,grid=True,label="g")
ccdVisitTable_r.plot(x="Time",y="zeroPoint",marker='+',c="r",lw=0.0,ax=ax1,grid=True,label="r")
ccdVisitTable_i.plot(x="Time",y="zeroPoint",marker='+',c="orange",lw=0.0,ax=ax1,grid=True,label="i")
#ccdVisitTable_z.plot(x="Time",y="zeroPoint",marker='+',c="grey",lw=0.0,ax=ax1,grid=True,label="z")
#ccdVisitTable_g.plot(x="Time",y="zeroPoint",marker='+',c="k",lw=0.0,ax=ax1,grid=True,label="y")
#ax1.set_title("zeroPoint")
ax1.set_ylabel("zeroPoint")
ax1.xaxis.set_major_formatter(date_form)
#ax1_up = ax1.secondary_xaxis("top", functions=( dt_to_mjd, mjd_to_dt))
#ccdVisitTable_g.plot(x="expMidptMJD",y="zeroPoint",marker='+',c="g",lw=0.0,ax=ax1_up,grid=True,label="g")


ccdVisitTable_g.plot(x="Time",y="airmass",marker='+',c="g",lw=0.0,ax=ax2,grid=True,label="g")
ccdVisitTable_r.plot(x="Time",y="airmass",marker='+',c="r",lw=0.0,ax=ax2,grid=True,label="r")
ccdVisitTable_i.plot(x="Time",y="airmass",marker='+',c="orange",lw=0.0,ax=ax2,grid=True,label="i")
#ccdVisitTable_z.plot(x="Time",y="airmass",marker='+',c="grey",lw=0.0,ax=ax2,grid=True,label="z")
#ccdVisitTable_y.plot(x="Time",y="airmass",marker='+',c="k",lw=0.0,ax=ax2,grid=True,label="y")
#ax2.set_title("airmass")
ax2.set_ylabel("airmass")
ax2.xaxis.set_major_formatter(date_form)

ccdVisitTable_g.plot(x="Time",y="seeing",marker='+',c="g",lw=0.0,ax=ax3,grid=True,label="g")
ccdVisitTable_r.plot(x="Time",y="seeing",marker='+',c="r",lw=0.0,ax=ax3,grid=True,label="r")
ccdVisitTable_i.plot(x="Time",y="seeing",marker='+',c="orange",lw=0.0,ax=ax3,grid=True,label="i")
#ccdVisitTable_z.plot(x="Time",y="seeing",marker='+',c="grey",lw=0.0,ax=ax3,grid=True,label="z")
#ccdVisitTable_y.plot(x="Time",y="seeing",marker='+',c="k",lw=0.0,ax=ax3,grid=True,label="y")
#ax2.set_title("airmass")
ax3.set_ylabel("seeing")
ax3.xaxis.set_major_formatter(date_form)

#df.plot(x="Time",y="coord_dec",marker='o',c="g",lw=0.0,ax=ax2,grid=True)
plt.suptitle(suptitle,y=1.0,fontsize=16)
plt.tight_layout()



In [ ]:
fig,axs = plt.subplots(3,1,figsize=(10,16))
ax1,ax2,ax3 = axs
ccdVisitTable_g.plot(x="airmass",y="zeroPoint",marker='+',c="g",lw=0.0,ax=ax1,grid=True,label="g")
ccdVisitTable_r.plot(x="airmass",y="zeroPoint",marker='+',c="r",lw=0.0,ax=ax1,grid=True,label="r")
ccdVisitTable_i.plot(x="airmass",y="zeroPoint",marker='+',c="orange",lw=0.0,ax=ax1,grid=True,label="i")
#ccdVisitTable_z.plot(x="airmass",y="zeroPoint",marker='+',c="grey",lw=0.0,ax=ax1,grid=True,label="z")
#ccdVisitTable_y.plot(x="airmass",y="zeroPoint",marker='+',c="k",lw=0.0,ax=ax1,grid=True,label="y")
#ax1.set_title("zeroPoint")
ax1.set_ylabel("zeroPoint vs airmass")
#ax1_up = ax1.secondary_xaxis("top", functions=( dt_to_mjd, mjd_to_dt))
#ccdVisitTable_g.plot(x="expMidptMJD",y="zeroPoint",marker='+',c="g",lw=0.0,ax=ax1_up,grid=True,label="g")

ccdVisitTable_g.plot(x="seeing",y="zeroPoint",marker='+',c="g",lw=0.0,ax=ax2,grid=True,label="g")
ccdVisitTable_r.plot(x="seeing",y="zeroPoint",marker='+',c="r",lw=0.0,ax=ax2,grid=True,label="r")
ccdVisitTable_i.plot(x="seeing",y="zeroPoint",marker='+',c="orange",lw=0.0,ax=ax2,grid=True,label="i")
#ccdVisitTable_z.plot(x="seeing",y="zeroPoint",marker='+',c="grey",lw=0.0,ax=ax2,grid=True,label="z")
#ccdVisitTable_y.plot(x="seeing",y="zeroPoint",marker='+',c="k",lw=0.0,ax=ax2,grid=True,label="y")
#ax2.set_title("airmass")
ax2.set_ylabel("zero point vs seeing")

ccdVisitTable_g.plot(x="airmass",y="seeing",marker='+',c="g",lw=0.0,ax=ax3,grid=True,label="g")
ccdVisitTable_r.plot(x="airmass",y="seeing",marker='+',c="r",lw=0.0,ax=ax3,grid=True,label="r")
ccdVisitTable_i.plot(x="airmass",y="seeing",marker='+',c="orange",lw=0.0,ax=ax3,grid=True,label="i")
#ccdVisitTable_z.plot(x="airmass",y="seeing",marker='+',c="grey",lw=0.0,ax=ax3,grid=True,label="z")
#ccdVisitTable_y.plot(x="airmass",y="seeing",marker='+',c="k",lw=0.0,ax=ax3,grid=True,label="y")
#ax2.set_title("airmass")
ax3.set_ylabel("seeing vs airmass")

#df.plot(x="Time",y="coord_dec",marker='o',c="g",lw=0.0,ax=ax2,grid=True)
plt.suptitle(suptitle,y=1.0,fontsize=16)
plt.tight_layout()

## Come-back to dataJoined

In [ ]:
dataJoined.columns

### recalculate sources properties inside dataJoined before adding CCDVisit info

In [ ]:
# Add columns into dataJoined table to have them all in one place
dataJoined["psfSn"] = dataJoined["psfFlux"]/dataJoined["psfFluxErr"]
dataJoined["psfMag"] = (dataJoined["psfFlux"].values*u.nJy).to(u.ABmag).value
dataJoined["psfMagErr"] = 2.5/np.log(10.0)*(dataJoined["psfFluxErr"].values/dataJoined["psfFlux"].values)

#dataJoined["apFlux_35_0_calFlux"] = dataJoined["apFlux_35_0_instFlux"]*dataJoined["localPhotoCalib"]
#dataJoined["apFlux_35_0_calFluxErr"] = dataJoined["apFlux_35_0_instFluxErr"]*dataJoined["localPhotoCalib"]
# This is the way to apply aperture corrections :
dataJoined[calibFluxCalStr] = dataJoined[calibFluxStr]*dataJoined["localPhotoCalib"]
dataJoined[calibFluxCalErrStr] = dataJoined[calibFluxErrStr]*dataJoined["localPhotoCalib"]

#dataJoined[calibFluxStr] = (dataJoined[calibFluxStr].values*u.nJy).to(u.ABmag).value
#dataJoined[calibFluxStr] = 2.5/np.log(10.0)*(dataJoined[calibFluxStr].values/dataJoined[calibFluxStr].values)
dataJoined[calibFluxCalMagStr] = (dataJoined[calibFluxCalStr].values*u.nJy).to(u.ABmag).value
dataJoined[calibFluxCalMagErrStr] = 2.5/np.log(10.0)*(dataJoined[calibFluxCalErrStr].values/dataJoined[calibFluxCalStr].values)

# NOTE: psfFlux is the fgcm calibrated flux.  I'm pretty sure you get the "instrumental" 
# flux by dividing psfFlux by the localPhotoCalib value.
dataJoined["psfInstMag"] = ((dataJoined["psfFlux"].values/dataJoined["localPhotoCalib"].values)*u.nJy).to(u.ABmag).value
dataJoined["psfGausFluxRatio"] = dataJoined["psfFlux"]/dataJoined["gaussianFlux"]

### Extract info from CCDVisit

In [ ]:
# Example of apply function format  that does what we expect
# dataJoined[["ra","dec"]].apply(lambda x: pd.Series([x["ra"], x["dec"]], index=['ra_2', 'dec_2']), axis=1)

In [ ]:
# Example of function calculating in which detector is the source
def findCCDVisitPerSource(x):
    """
    Find the CCD visit cooresponding to that source
    """
    visitId = x["visit"]
    ra0 = x["ra"]
    dec0 = x ["dec"]
    ccd_visit_table_forthatvisit = ccdVisitTable[ccdVisitTable.visitId == visitId]
    ccd_visit_table_forthatvisit["select"] =  ccd_visit_table_forthatvisit.apply(lambda x :  is_in_polygon(ra0,dec0,(x["llcra"],x["llcdec"]), (x["lrcra"],x["lrcdec"]), (x["urcra"],x["urcdec"]),(x["ulcra"],x["ulcdec"])),axis=1)
    ccd_visit_table_forthatvisit = ccd_visit_table_forthatvisit[ccd_visit_table_forthatvisit["select"]] 
    ccd_visit_table_forthatvisit["dra"] = ra0 - ccd_visit_table_forthatvisit["ra"] 
    ccd_visit_table_forthatvisit["ddec"] = dec0 - ccd_visit_table_forthatvisit["dec"] 
    ccd_visit_table_forthatvisit.reset_index(inplace=True)
    ccd_visit_table_forthatvisit.rename(columns = {'ccdVisitId':'ccd_ccdVisitId',"visitId": "ccd_visitId","band":"ccd_band","detector":"ccd_detector"}, inplace = True) 
    ccd_visit_table_forthatvisit.drop(columns = ["physical_filter","ra","dec","select"],inplace=True)
    
    assert len(ccd_visit_table_forthatvisit == 1)
    # Convert the dataframe into a pandas series
    # Do not do ccd_visit_table_forthatvisit.T
    ser =  ccd_visit_table_forthatvisit.iloc[0,:]
    ser.name = "ccd_visit_table_forthatvisit"
    return ser
    

In [ ]:
dataJoined_subset = dataJoined.iloc[:10]
dataJoined_subset_toadd = dataJoined_subset.apply(findCCDVisitPerSource,axis=1)
dataJoined_subset_toadd

In [ ]:
dataJoined_subset_toadd

In [ ]:
dataJoined_subset.join(dataJoined_subset_toadd) 

### Extract the columns from the CCDVisitTable corresponding to each row of dataJoined

In [ ]:
dataJoined_toadd = dataJoined.apply(findCCDVisitPerSource,axis=1)

In [ ]:
dataJoined_toadd

In [ ]:
dataJoined = dataJoined.join(dataJoined_toadd) 

In [ ]:
dataJoined.plot.scatter(x="detector",y="ccd_detector") 

In [ ]:
dataJoined.plot.scatter(x="visit",y="ccd_visitId") 

In [ ]:
# Add columns into dataJoined table to have them all in one place
#dataJoined["psfSn"] = dataJoined["psfFlux"]/dataJoined["psfFluxErr"]
#dataJoined["psfMag"] = (dataJoined["psfFlux"].values*u.nJy).to(u.ABmag).value
#dataJoined["psfMagErr"] = 2.5/np.log(10.0)*(dataJoined["psfFluxErr"].values/dataJoined["psfFlux"].values)

##dataJoined["apFlux_35_0_calFlux"] = dataJoined["apFlux_35_0_instFlux"]*dataJoined["localPhotoCalib"]
##dataJoined["apFlux_35_0_calFluxErr"] = dataJoined["apFlux_35_0_instFluxErr"]*dataJoined["localPhotoCalib"]
# This is the way to apply aperture corrections :
#dataJoined[calibFluxCalStr] = dataJoined[calibFluxStr]*dataJoined["localPhotoCalib"]
#dataJoined[calibFluxCalErrStr] = dataJoined[calibFluxErrStr]*dataJoined["localPhotoCalib"]

#dataJoined[calibFluxStr] = (dataJoined[calibFluxStr].values*u.nJy).to(u.ABmag).value
#dataJoined[calibFluxStr] = 2.5/np.log(10.0)*(dataJoined[calibFluxStr].values/dataJoined[calibFluxStr].values)
#dataJoined[calibFluxCalMagStr] = (dataJoined[calibFluxCalStr].values*u.nJy).to(u.ABmag).value
#dataJoined[calibFluxCalMagErrStr] = 2.5/np.log(10.0)*(dataJoined[calibFluxCalErrStr].values/dataJoined[calibFluxCalStr].values)

# NOTE: psfFlux is the fgcm calibrated flux.  I'm pretty sure you get the "instrumental" 
# flux by dividing psfFlux by the localPhotoCalib value.
#dataJoined["psfInstMag"] = ((dataJoined["psfFlux"].values/dataJoined["localPhotoCalib"].values)*u.nJy).to(u.ABmag).value
#dataJoined["psfGausFluxRatio"] = dataJoined["psfFlux"]/dataJoined["gaussianFlux"]

#dataJoined["psfSigma"] = dataJoined.apply(lambda x: psfSigmaDict[x["visit"]], axis=1)
#dataJoined["skyBg"] = dataJoined.apply(lambda x: skyBgDict[x["visit"]], axis=1)
#dataJoined["expTime"] = dataJoined.apply(lambda x: expTimeDict[x["visit"]], axis=1)
#dataJoined["expMidptMjd"] = dataJoined.apply(lambda x: expMidptMjdDict[x["visit"]], axis=1)
#dataJoined["nSource"] = dataJoined.apply(lambda x: nSourceDict[x["visit"]], axis=1)
#dataJoined["medianE"] = dataJoined.apply(lambda x: medianEDict[x["visit"]], axis=1)
#dataJoined["psfStarScaledDeltaSizeScatter"] = dataJoined.apply(lambda x: psfStarScaledDeltaSizeScatterDict[x["visit"]], axis=1)
#dataJoined["astromOffsetStd"] = dataJoined.apply(lambda x: astromOffsetStdDict[x["visit"]], axis=1)
#dataJoined["psfTraceRadiusDelta"] = dataJoined.apply(lambda x: psfTraceRadiusDeltaDict[x["visit"]], axis=1)

# I add the airmass and zero-point I want to keep here
#dataJoined["zeroPoint"] = dataJoined.apply(lambda x: zeroPointDict[x["visit"]], axis=1)
#dataJoined["localPhotoCalib"] = dataJoined.apply(lambda x: localPhotoCalibDict[x["visit"]], axis=1)
#dataJoined["airmass"] = dataJoined.apply(lambda x: airmassDict[x["visit"]], axis=1)
#dataJoined["seeing"] = dataJoined.apply(lambda x: seeingDict[x["visit"]], axis=1)
#dataJoined["DZP"] = dataJoined.apply(lambda x: DZPDict[x["visit"]], axis=1)

#matchedObjIdList = list(set(dataJoined["obj_index"]))

In [ ]:
matchedObjIdList = list(set(dataJoined["obj_index"]))

In [ ]:
dataJoined[dataJoined.index == 0]

## The association between the independent sources is done through the object index 

- for plotting the repeatability

In [ ]:
# Compute repeatability related values per object/object group and add
# them inplace to the dataJoined DataFrame.
psfMagDiffMmagDfList = []
psfMagStdMmagAll = []
psfMagStdMmagDict = {}
psfMagDiffChiDfList = []
psfSnAll = []
raDiffMasDfList = []
decDiffMasDfList = []
raCosDecDiffMasDfList = []
objRaDegList = []
objDecDegList = []
raStdMasAll = []
decStdMasAll = []
raCosDecStdMasAll = []
objDataList = []

# loop on objects
for objIndex in matchedObjIdList:
    # select sources corresponding to that object
    objData = dataJoined[dataJoined["obj_index"] == objIndex]
    psfMag = objData["psfMag"]
    psfMagErr = objData["psfMagErr"]

    # average flux over all visits
    clippedMeanMagArray = scipy.stats.sigmaclip(psfMag, low=3.0, high=3.0).clipped
    clippedMeanMag = np.mean(clippedMeanMagArray)
    clippedMeanMagErr = np.std(clippedMeanMagArray)/np.sqrt(len(clippedMeanMagArray))

    # array of difference between flux an clipped mean average
    psfMagDiffMmag = 1000*(psfMag - clippedMeanMag)
    # sigma arrays over the difference
    psfMagStdMmag = np.std(psfMagDiffMmag)
    # residuals array
    psfMagDiffChi = (psfMag - clippedMeanMag)/np.sqrt(psfMagErr**2 + clippedMeanMagErr**2)
    
    clippedMeanRaArray = scipy.stats.sigmaclip(objData["ra"], low=3.0, high=3.0).clipped
    clippedMeanRa = np.mean(clippedMeanRaArray)
    raDiffMas = (objData["ra"] - clippedMeanRa)*3600*1000
    
    clippedMeanDecArray = scipy.stats.sigmaclip(objData["dec"], low=3.0, high=3.0).clipped
    clippedMeanDec = np.mean(clippedMeanDecArray)
    decDiffMas = (objData["dec"] - clippedMeanDec)*3600*1000
    
    raCosDec = np.deg2rad(objData["ra"])*np.cos(np.deg2rad(objData["dec"]))
    clippedMeanRaCosDecArray = scipy.stats.sigmaclip(raCosDec, low=3.0, high=3.0).clipped
    clippedMeanRaCosDec = np.mean(clippedMeanRaCosDecArray)
    raCosDecDiffRad = (raCosDec - clippedMeanRaCosDec)
    raCosDecDiffMas = np.rad2deg(raCosDecDiffRad)*3600*1000

    # save for all sources of the object
    psfMagDiffMmagDfList.append(psfMagDiffMmag)
    psfMagStdMmagAll.append(psfMagStdMmag)
    psfMagDiffChiDfList.append(psfMagDiffChi)

    raDiffMasDfList.append(raDiffMas)
    decDiffMasDfList.append(decDiffMas)
    raCosDecDiffMasDfList.append(raCosDecDiffMas)
    
    objRaDegList.append(clippedMeanRa)
    objDecDegList.append(clippedMeanDec)

    raStdMas = np.std(raDiffMas)
    raStdMasAll.append(raStdMas)
    decStdMas = np.std(decDiffMas)
    decStdMasAll.append(decStdMas)
    raCosDecStdMas = np.std(raCosDecDiffMas)
    raCosDecStdMasAll.append(raCosDecStdMas)
    psfMagStdMmagDict[objIndex] = psfMagStdMmag

# concatenate over all objects - all sources
psfMagDiffMmagDf = pd.concat(psfMagDiffMmagDfList)
psfMagDiffChiDf = pd.concat(psfMagDiffChiDfList)
raDiffMasDf = pd.concat(raDiffMasDfList)
decDiffMasDf = pd.concat(decDiffMasDfList)
raCosDecDiffMasDf = pd.concat(raCosDecDiffMasDfList)

# add this common properties to each source in dataJoined
dataJoined.loc[:, "psfMagDiffMmag"] = psfMagDiffMmagDf
dataJoined.loc[:, "psfMagDiffChi"] = psfMagDiffChiDf
dataJoined.loc[:, "raDiffMas"] = raDiffMasDf
dataJoined.loc[:, "decDiffMas"] = decDiffMasDf
dataJoined.loc[:, "raCosDecDiffMas"] = raCosDecDiffMasDf

In [ ]:
if 0:
    # Compute repeatability related values per object/object group and add
    # them inplace to the dataJoined DataFrame.
    psfMagDiffMmagDfList = []
    psfMagStdMmagAll = []
    psfMagStdMmagDict = {}
    psfMagDiffChiDfList = []
    psfSnAll = []
    raDiffMasDfList = []
    decDiffMasDfList = []
    raCosDecDiffMasDfList = []
    objRaDegList = []
    objDecDegList = []
    raStdMasAll = []
    decStdMasAll = []
    raCosDecStdMasAll = []
    objDataList = []

    # loop on objects
    for objIndex in matchedObjIdList:
        # select sources corresponding to that object
        objData = dataJoined[dataJoined["obj_index"] == objIndex]
        psfMag = objData["psfMag"]
        psfMagErr = objData["psfMagErr"]

        # average flux over all visits
        clippedMeanMagArray = scipy.stats.sigmaclip(psfMag, low=3.0, high=3.0).clipped
        clippedMeanMag = np.mean(clippedMeanMagArray)
        clippedMeanMagErr = np.std(clippedMeanMagArray)/np.sqrt(len(clippedMeanMagArray))

        # array of difference between flux an clipped mean average
        psfMagDiffMmag = 1000*(psfMag - clippedMeanMag)
        # sigma arrays over the difference
        psfMagStdMmag = np.std(psfMagDiffMmag)
        # residuals array
        psfMagDiffChi = (psfMag - clippedMeanMag)/np.sqrt(psfMagErr**2 + clippedMeanMagErr**2)
    
        clippedMeanRaArray = scipy.stats.sigmaclip(objData["ra"], low=3.0, high=3.0).clipped
        clippedMeanRa = np.mean(clippedMeanRaArray)
        raDiffMas = (objData["ra"] - clippedMeanRa)*3600*1000
    
        clippedMeanDecArray = scipy.stats.sigmaclip(objData["dec"], low=3.0, high=3.0).clipped
        clippedMeanDec = np.mean(clippedMeanDecArray)
        decDiffMas = (objData["dec"] - clippedMeanDec)*3600*1000
    
        raCosDec = np.deg2rad(objData["ra"])*np.cos(np.deg2rad(objData["dec"]))
        clippedMeanRaCosDecArray = scipy.stats.sigmaclip(raCosDec, low=3.0, high=3.0).clipped
        clippedMeanRaCosDec = np.mean(clippedMeanRaCosDecArray)
        raCosDecDiffRad = (raCosDec - clippedMeanRaCosDec)
        raCosDecDiffMas = np.rad2deg(raCosDecDiffRad)*3600*1000

        # save for all sources of the object
        psfMagDiffMmagDfList.append(psfMagDiffMmag)
        psfMagStdMmagAll.append(psfMagStdMmag)
        psfMagDiffChiDfList.append(psfMagDiffChi)

        raDiffMasDfList.append(raDiffMas)
        decDiffMasDfList.append(decDiffMas)
        raCosDecDiffMasDfList.append(raCosDecDiffMas)
    
        objRaDegList.append(clippedMeanRa)
        objDecDegList.append(clippedMeanDec)

        raStdMas = np.std(raDiffMas)
        raStdMasAll.append(raStdMas)
        decStdMas = np.std(decDiffMas)
        decStdMasAll.append(decStdMas)
        raCosDecStdMas = np.std(raCosDecDiffMas)
        raCosDecStdMasAll.append(raCosDecStdMas)
        psfMagStdMmagDict[objIndex] = psfMagStdMmag

    # concatenate over all objects - all sources
    psfMagDiffMmagDf = pd.concat(psfMagDiffMmagDfList)
    psfMagDiffChiDf = pd.concat(psfMagDiffChiDfList)
    raDiffMasDf = pd.concat(raDiffMasDfList)
    decDiffMasDf = pd.concat(decDiffMasDfList)
    raCosDecDiffMasDf = pd.concat(raCosDecDiffMasDfList)

    # add this common properties to each source in dataJoined
    dataJoined.loc[:, "psfMagDiffMmag"] = psfMagDiffMmagDf
    dataJoined.loc[:, "psfMagDiffChi"] = psfMagDiffChiDf
    dataJoined.loc[:, "raDiffMas"] = raDiffMasDf
    dataJoined.loc[:, "decDiffMas"] = decDiffMasDf
    dataJoined.loc[:, "raCosDecDiffMas"] = raCosDecDiffMasDf

In [ ]:
dataJoined[dataJoined.index == 0]

In [ ]:
dataJoined.head()

## Check the correlation between the flux psf and apperture flux

### Compute DPZ

In [ ]:
dataJoined["localPhotoCalibMag"] = (dataJoined["localPhotoCalib"].values*u.nJy).to(u.ABmag).value
dataJoined["localPhotoCalibMagErr"] =   2.5/np.log(10.0)*dataJoined["localPhotoCalibErr"].values/dataJoined["localPhotoCalib"].values
dataJoined["DZP"] = dataJoined["localPhotoCalibMag"] - dataJoined["zeroPoint"]

- The goal is to understand what flux is calibrated what flux is not calibrated 

In [ ]:
dataJoined_subset = dataJoined[["obj_index","sourceId","band","visit","ra","dec","psfMag","psfFlux","gaussianFlux",calibFluxCalStr,calibFluxCalMagStr,calibFluxStr,"psfFlux_apCorr",
                                "psfMagErr","psfFluxErr","gaussianFluxErr",calibFluxCalErrStr,calibFluxCalMagErrStr,calibFluxErrStr,"skyBg","zeroPoint","localPhotoCalib","localPhotoCalibMag","DZP","localPhotoCalibMagErr","airmass","seeing","detector","x","y"]].copy()

In [ ]:
dataJoined_subset.sort_values(by=["obj_index","band","visit"])

In [ ]:
dataJoined_subset_g = dataJoined_subset[dataJoined_subset["band"] == "g"]
dataJoined_subset_r = dataJoined_subset[dataJoined_subset["band"] == "r"]
dataJoined_subset_i = dataJoined_subset[dataJoined_subset["band"] == "i"]
dataJoined_subset_z = dataJoined_subset[dataJoined_subset["band"] == "z"]
dataJoined_subset_y = dataJoined_subset[dataJoined_subset["band"] == "y"]

### Histogram on calibration

In [ ]:
all_dpz_std = []
all_labels = []
fig,ax = plt.subplots(1,1,figsize=(8,6))

std = dataJoined_subset_g["DZP"].std()*1000
label = f"g, sig(DPZ) = {std:.2f} mmag"
all_dpz_std.append(std)
all_labels.append(label) 
dataJoined_subset_g["DZP"].plot.hist(bins=50,histtype="step",density=True,ax=ax,color="g",lw=2,label=label)

std = dataJoined_subset_r["DZP"].std()*1000
label = f"r, sig(DPZ) = {std:.2f} mmag"
all_dpz_std.append(std)
all_labels.append(label) 
dataJoined_subset_r["DZP"].plot.hist(bins=50,histtype="step",density=True,ax=ax,color="r",lw=2,label=label)

std = dataJoined_subset_i["DZP"].std()*1000
label = f"i, sig(DPZ) = {std:.2f} mmag"
all_dpz_std.append(std)
all_labels.append(label) 
dataJoined_subset_i["DZP"].plot.hist(bins=50,histtype="step",density=True,ax=ax,color="orange",lw=2,label=label)

std = dataJoined_subset_z["DZP"].std()*1000
label = f"z, sig(DPZ) = {std:.2f} mmag"
all_dpz_std.append(std)
all_labels.append(label) 
dataJoined_subset_z["DZP"].plot.hist(bins=50,histtype="step",density=True,ax=ax,color="grey",lw=2,label=label)

std = dataJoined_subset_y["DZP"].std()*1000
label = f"y, sig(DPZ) = {std:.2f} mmag"
all_dpz_std.append(std)
all_labels.append(label) 
dataJoined_subset_y["DZP"].plot.hist(bins=50,histtype="step",density=True,ax=ax,color="k",lw=2,label=label)

ax.set_xlabel("DZP : localPhotoCalib - zeroPoint (mag)")
ax.set_xlim(-0.01,0.01)
ax.grid()
ax.legend()
ax.set_title(f"Photometric calibration parameters: localPhotoCalib(sourceTable)-PZ(CCDvisitTable) tract {tract}",fontsize=12)
plt.suptitle(suptitle,y=1.0,fontsize=8)
plt.tight_layout()

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(18,6),sharex=True,sharey=True)
#ax1,ax2,ax3,ax4,ax5 =axs
ax1,ax2,ax3 =axs
dataJoined_subset_g.plot.scatter(x="psfMag",y="DZP",ax=ax1,color="g",marker="+",lw=1,label="g",title=all_labels[0],grid=True,alpha=0.5)
dataJoined_subset_r.plot.scatter(x="psfMag",y="DZP",ax=ax2,color="r",marker="+",lw=1,label="r",title=all_labels[1],grid=True,alpha=0.5)
dataJoined_subset_i.plot.scatter(x="psfMag",y="DZP",ax=ax3,color="orange",marker="+",lw=1,label="i",title=all_labels[2],grid=True,alpha=0.5)
#dataJoined_subset_z.plot.scatter(x="psfMag",y="DZP",ax=ax4,color="grey",marker="+",lw=0.5,label="z",title=all_labels[3],grid=True,alpha=0.5)
#dataJoined_subset_y.plot.scatter(x="psfMag",y="DZP",ax=ax5,color="k",marker="+",lw=0.5,label="y",title=all_labels[4],grid=True,alpha=0.5)
ax1.set_ylabel("DPZ (mag)")
plt.suptitle(suptitle,y=1.0,fontsize=8)
plt.tight_layout()

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(18,6),sharex=True,sharey=True)
#ax1,ax2,ax3,ax4,ax5 =axs
ax1,ax2,ax3 =axs
dataJoined_subset_g.plot.scatter(x="airmass",y="DZP",ax=ax1,color="g",marker="+",lw=1,label="g",title=all_labels[0],grid=True,alpha=0.5)
dataJoined_subset_r.plot.scatter(x="airmass",y="DZP",ax=ax2,color="r",marker="+",lw=1,label="r",title=all_labels[1],grid=True,alpha=0.5)
dataJoined_subset_i.plot.scatter(x="airmass",y="DZP",ax=ax3,color="orange",marker="+",lw=1,label="i",title=all_labels[2],grid=True,alpha=0.5)
#dataJoined_subset_z.plot.scatter(x="airmass",y="DZP",ax=ax4,color="grey",marker="+",lw=0.5,label="z",title=all_labels[3],grid=True,alpha=0.5)
#dataJoined_subset_y.plot.scatter(x="airmass",y="DZP",ax=ax5,color="k",marker="+",lw=0.5,label="y",title=all_labels[4],grid=True,alpha=0.5)
ax1.set_ylabel("DPZ (mag)")
plt.suptitle(suptitle,y=1.0,fontsize=8)
plt.tight_layout()

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(18,6),sharex=True,sharey=True)
#ax1,ax2,ax3,ax4,ax5 =axs
ax1,ax2,ax3 = axs
dataJoined_subset_g.plot.scatter(x="visit",y="DZP",ax=ax1,color="g",marker="+",lw=1,label="g",title=all_labels[0],grid=True,alpha=0.5)
dataJoined_subset_r.plot.scatter(x="visit",y="DZP",ax=ax2,color="r",marker="+",lw=1,label="r",title=all_labels[1],grid=True,alpha=0.5)
dataJoined_subset_i.plot.scatter(x="visit",y="DZP",ax=ax3,color="orange",marker="+",lw=1,label="i",title=all_labels[2],grid=True,alpha=0.5)
#dataJoined_subset_z.plot.scatter(x="visit",y="DZP",ax=ax4,color="grey",marker="+",lw=0.5,label="z",title=all_labels[3],grid=True,alpha=0.5)
#dataJoined_subset_y.plot.scatter(x="visit",y="DZP",ax=ax5,color="k",marker="+",lw=0.5,label="y",title=all_labels[4],grid=True,alpha=0.5)
ax1.set_ylabel("DPZ (mag)")
plt.suptitle(suptitle,y=1.0,fontsize=8)
plt.tight_layout()

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(18,6),sharex=True,sharey=True)
#ax1,ax2,ax3,ax4,ax5 =axs
ax1,ax2,ax3 =axs
dataJoined_subset_g.plot.scatter(x="detector",y="DZP",ax=ax1,color="g",marker="o",lw=1,label="g",title=all_labels[0],grid=True,alpha=0.5)
dataJoined_subset_r.plot.scatter(x="detector",y="DZP",ax=ax2,color="r",marker="o",lw=1,label="r",title=all_labels[1],grid=True,alpha=0.5)
dataJoined_subset_i.plot.scatter(x="detector",y="DZP",ax=ax3,color="orange",marker="o",lw=1,label="i",title=all_labels[2],grid=True,alpha=0.5)
#dataJoined_subset_z.plot.scatter(x="detector",y="DZP",ax=ax4,color="grey",marker="o",lw=0.5,label="z",title=all_labels[3],grid=True,alpha=0.5)
#dataJoined_subset_y.plot.scatter(x="detector",y="DZP",ax=ax5,color="k",marker="o",lw=0.5,label="y",title=all_labels[4],grid=True,alpha=0.5)
ax1.set_ylabel("DPZ (mag)")
plt.suptitle(suptitle,y=1.0,fontsize=8)
plt.tight_layout()

In [ ]:
all_localPhotoCalibMagErr_aver = []
all_labels = []

aver = dataJoined_subset_g["localPhotoCalibMagErr"].mean()*1000
label = f"g, aver(locPCalErr) = {aver:.2f} mmag"
all_localPhotoCalibMagErr_aver.append(std)
all_labels.append(label) 

aver = dataJoined_subset_r["localPhotoCalibMagErr"].mean()*1000
label = f"r, aver(locPCalErr) = {aver:.2f} mmag"
all_localPhotoCalibMagErr_aver.append(std)
all_labels.append(label) 

aver = dataJoined_subset_i["localPhotoCalibMagErr"].mean()*1000
label = f"i, aver(locPCalErr) = {aver:.2f} mmag"
all_localPhotoCalibMagErr_aver.append(std)
all_labels.append(label) 

aver = dataJoined_subset_z["localPhotoCalibMagErr"].mean()*1000
label = f"z, aver(locPCalErr) = {aver:.2f} mmag"
all_localPhotoCalibMagErr_aver.append(std)
all_labels.append(label) 

aver = dataJoined_subset_y["localPhotoCalibMagErr"].mean()*1000
label = f"y, aver(locPCalErr) = {aver:.2f} mmag"
all_localPhotoCalibMagErr_aver.append(std)
all_labels.append(label) 

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(18,4),sharex=True,sharey=True)
#ax1,ax2,ax3,ax4,ax5 =axs
ax1,ax2,ax3 =axs
dataJoined_subset_g.plot.scatter(x="psfMag",y="localPhotoCalibMagErr",ax=ax1,color="g",marker="+",lw=1,label="g",title=all_labels[0],grid=True,alpha=0.5)
dataJoined_subset_r.plot.scatter(x="psfMag",y="localPhotoCalibMagErr",ax=ax2,color="r",marker="+",lw=1,label="r",title=all_labels[1],grid=True,alpha=0.5)
dataJoined_subset_i.plot.scatter(x="psfMag",y="localPhotoCalibMagErr",ax=ax3,color="orange",marker="+",lw=1,label="i",title=all_labels[2],grid=True,alpha=0.5)
#dataJoined_subset_z.plot.scatter(x="psfMag",y="localPhotoCalibMagErr",ax=ax4,color="grey",marker="+",lw=0.5,label="z",title=all_labels[3],grid=True,alpha=0.5)
#dataJoined_subset_y.plot.scatter(x="psfMag",y="localPhotoCalibMagErr",ax=ax5,color="k",marker="+",lw=0.5,label="y",title=all_labels[4],grid=True,alpha=0.5)
ax1.set_ylabel("localPhotoCalibMagErr (mag)")
ax1.set_ylim(0.,0.001)
plt.suptitle(suptitle,y=1.0,fontsize=8)
plt.tight_layout()

In [ ]:
NDET

In [ ]:
all_det_colors

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(18,5),sharex=True,sharey=False)
#ax1,ax2,ax3,ax4,ax5 =axs
ax1,ax2,ax3 =axs
for idet in range(NDET):
    select_cut = dataJoined_subset_g["detector"] == idet
    the_color = all_det_colors[idet]
    the_label = f"det = {idet}"
    the_title = f"band g , tract {tract}"
    the_data = dataJoined_subset_g[select_cut]
    the_data.plot.scatter(x="airmass",y="localPhotoCalibMag",ax=ax1,color=the_color,marker="o",lw=1,label=the_label,title=the_title,grid=True,alpha=0.5)

for idet in range(NDET):
    select_cut = dataJoined_subset_r["detector"] == idet
    the_color = all_det_colors[idet]
    the_label = f"det = {idet},"
    the_title = f"band r , tract {tract}"
    the_data = dataJoined_subset_r[select_cut]
    the_data.plot.scatter(x="airmass",y="localPhotoCalibMag",ax=ax2,color=the_color,marker="o",lw=1,label=the_label,title=the_title,grid=True,alpha=0.5)

for idet in range(NDET):
    select_cut = dataJoined_subset_i["detector"] == idet
    the_color = all_det_colors[idet]
    the_label = f"det = {idet}"
    the_title = f"band i , tract {tract}"
    the_data = dataJoined_subset_i[select_cut]
    the_data.plot.scatter(x="airmass",y="localPhotoCalibMag",ax=ax3,color=the_color,marker="o",lw=1,label=the_label,title=the_title,grid=True,alpha=0.5)
    

#ax1.set_ylabel(f"localPhotoCalibMag (mag) for the different detectors (tract {tract})")
#ax1.set_ylim(0.,0.001)
plt.suptitle(suptitle,y=1.0,fontsize=10)
plt.tight_layout()

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(18,5),sharex=True,sharey=False)
#ax1,ax2,ax3,ax4,ax5 =axs
ax1,ax2,ax3 =axs
for idet in range(NDET):
    select_cut = dataJoined_subset_g["detector"] == idet
    the_color = all_det_colors[idet]
    the_label = f"det = {idet}"
    the_title = f"band g , tract {tract}"
    the_data = dataJoined_subset_g[select_cut]
    the_data.plot.scatter(x="airmass",y="zeroPoint",ax=ax1,color=the_color,marker="o",lw=1,label=the_label,title=the_title,grid=True,alpha=0.5)

for idet in range(NDET):
    select_cut = dataJoined_subset_r["detector"] == idet
    the_color = all_det_colors[idet]
    the_label = f"det = {idet}"
    the_title = f"band r , tract {tract}"
    the_data = dataJoined_subset_r[select_cut]
    the_data.plot.scatter(x="airmass",y="zeroPoint",ax=ax2,color=the_color,marker="o",lw=1,label=the_label,title=the_title,grid=True,alpha=0.5)

for idet in range(NDET):
    select_cut = dataJoined_subset_i["detector"] == idet
    the_color = all_det_colors[idet]
    the_label = f"det = {idet}"
    the_title = f"band i , tract {tract}"
    the_data = dataJoined_subset_i[select_cut]
    the_data.plot.scatter(x="airmass",y="zeroPoint",ax=ax3,color=the_color,marker="o",lw=1,label=the_label,title=the_title,grid=True,alpha=0.5)
    

#ax.set_title(f"zeroPoint (mag) for the different detectors (tract {tract})")
plt.suptitle(suptitle,y=1.0,fontsize=10)
plt.tight_layout()

## Non uniformity on CCD

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(18,4))
ax1,ax2,ax3 = axs[0], axs[1],axs[2]

std = dataJoined_subset_g["DZP"].std()*1000
label = f"g, sig(DPZ) = {std:.2f} mmag"
dataJoined_subset_g.plot.scatter(x="x",y="y",ax=ax1,marker="+",lw=1,c="DZP",cmap="seismic",label="g",title=label,grid=True,alpha=1,colorbar="True",vmin=-0.002,vmax=0.002)
ax1.set_aspect("equal")

std = dataJoined_subset_r["DZP"].std()*1000
label = f"r, sig(DPZ) = {std:.2f} mmag"
dataJoined_subset_r.plot.scatter(x="x",y="y",ax=ax2,marker="+",lw=1,c="DZP",cmap="seismic",label="r",title=label,grid=True,alpha=1,colorbar="True",vmin=-0.002,vmax=0.002)
ax2.set_aspect("equal")

std = dataJoined_subset_i["DZP"].std()*1000
label = f"i, sig(DPZ) = {std:.2f} mmag"
dataJoined_subset_i.plot.scatter(x="x",y="y",ax=ax3,marker="+",lw=1,c="DZP",cmap="seismic",label="i",title=label,grid=True,alpha=1,colorbar="True",vmin=-0.002,vmax=0.002)
ax3.set_aspect("equal")


plt.suptitle(suptitle)
plt.tight_layout()

In [ ]:
the_subset = dataJoined_subset_g
the_detector = the_subset["detector"].unique()
NDET = len(the_detector)

In [ ]:
fig,axes = plt.subplots(3,3,figsize=(16,13))
for i, ax in enumerate(axes.flat):
    the_detector_subset = the_subset[the_subset["detector"] == i ]
    std = the_detector_subset["DZP"].std()*1000
    label = f"g,{i} sig(DPZ) = {std:.2f} mmag"
    the_detector_subset.plot.scatter(x="x",y="y",ax=ax,marker="+",lw=1,c="DZP",cmap="seismic",label="g",title=label,grid=True,alpha=1,colorbar="True",vmin=-0.002,vmax=0.002)
    ax.set_aspect("equal")


plt.suptitle("g band multi-detector :"+suptitle)
plt.tight_layout()